In [ ]:
import tensorflow
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.layers import Activation
from keras.layers import Dropout, Flatten, Dense, Reshape
from keras import backend as K

import os
import time
import numpy as np
import matplotlib
from tensorflow import keras


# In[2]:


input_size = 360,720

# import cv2

# cv2.imread('C:/Users/HP/Desktop/Pneumonia project dataset and code/Final data of project/Dataset after augmentation/Training_dataset/Healthy_data_Result/102_1b1_Ar_sc_Meditron.png')


train_data_dir = 'C:/Users/HP/Desktop/Italian_Dataset_Melspectrogram/E/Training'
test_data_dir = 'C:/Users/HP/Desktop/Italian_Dataset_Melspectrogram/E/Testing'


epochs = 15
batch_size = 32


# In[3]:


from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1. /255,
    width_shift_range=0.2, # Shift the image horizontally by 20% of the width
    height_shift_range=0.2, # Shift the image vertically by 20% of the height
    zoom_range=0.2, # Zoom in the image by 20%
    fill_mode='nearest' # Fill any empty pixels with the nearest available pixel
)

test_datagen = ImageDataGenerator(rescale=1. /255 )
   
    
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(360, 720),                
    batch_size=batch_size,
    class_mode='binary')    


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(360, 720),
    batch_size=batch_size,
    class_mode='binary')


# In[4]:


y_true= test_generator.classes
print(y_true)


# In[5]:


keras.backend.clear_session()


import tensorflow as tf

# Set hyperparameters

CLASSES = ['Healthy', 'Pneumonia']
input_shape= (100,200)
num_epochs = 15
batch_size = 32
lstm_units = 128
dropout_rate = 0.2

# Define model architecture
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Reshape((64,178*88)))
model.add(LSTM(units=lstm_units, dropout=dropout_rate, return_sequences=True))
model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



# Train model
model.fit(
    train_generator, 

    epochs=num_epochs,
    batch_size=batch_size)



model.summary()


# In[6]:



loss, accuracy = model.evaluate(train_generator)
print("Train loss:", loss)
print("Train accuracy:", accuracy)


# In[7]:



loss, accuracy = model.evaluate(test_generator)
print("Test loss:", loss)
print("Test accuracy:", accuracy)


# In[8]:


model.save("my_model_cnn_lstm.h5")


# In[9]:


y_pred=(model.predict(test_generator)>0.5).astype(int)



import matplotlib.pyplot as plt

import numpy

from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_true, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels

= [False, True])

cm_display.plot()

plt.show()


# In[10]:


from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

print('Accuracy: %.3f' % accuracy_score(y_true, y_pred))
print('precision_score: %.3f' % precision_score(y_true, y_pred))
print(' recall_score: %.3f' %  recall_score(y_true, y_pred))
print('f1_score: %.3f' % f1_score(y_true, y_pred))
#print('MCC_score: %, 3f' % matthews_corrcoef(y_true, y_pred))


# In[11]:


from sklearn.metrics import matthews_corrcoef
print('MCC_score: %.3f' % matthews_corrcoef(y_true, y_pred))
#print('MCC_score: %, 3f' % matthews_corrcoef(y_true, y_pred))


# In[12]:


import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['test accuracy']
loss = history.history['loss']
val_loss = history.history['test loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()